# Prepare Ribbon for SPM Simulation with AiiDA

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
  
from IPython.display import display, clear_output
from collections import OrderedDict
import ipywidgets as ipw
import nglview

## Step 1: Select a structure from the AiiDA database

In [ ]:
# query AiiDA database for structures
qb = QueryBuilder()
qb.append(StructureData)
   
all_nodes = [ node_tuple[0] for node_tuple in qb.iterall() ]
all_nodes.sort(key=lambda n: n.pk)

all_structs = OrderedDict()
all_structs["Select a Structure"] = False

#print("Loading structures...")
for node in all_nodes:
    label = "%s PK: %d %s"%(node.get_formula(), node.pk, node.description)
    all_structs[label] = node

def on_struct_change(c):
    global atoms
    node = drop_struct.value
    atoms = node.get_ase() if node else None
    update_viewer()
    
drop_struct = ipw.Dropdown(options=all_structs, value=False)
drop_struct.observe(on_struct_change, names='value')
display(drop_struct)

## Step 2: Edit Structure

In [ ]:
def update_viewer():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    if atoms:
        viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
        viewer.add_unitcell()
        viewer.center_view()
        
        spin_ups = ",".join([str(i) for i, t in enumerate(atoms.get_tags()) if t==1])
        spin_downs = ",".join([str(i) for i, t in enumerate(atoms.get_tags()) if t==2])
        
        viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=3.0, opacity=0.4)
        viewer.add_representation('ball+stick', selection="@"+spin_downs, color='blue', aspectRatio=3.0, opacity=0.4)

In [ ]:
def on_picked(c):
    global atoms, viewer
        
    if 'atom' not in viewer.picked.keys():
        return # did not click on atom

    idx = viewer.picked['atom']['index']
    tags = atoms.get_tags()

    if tool.value == "spin 0":
        tags[idx] = 0
        atoms.set_tags(tags)
    elif tool.value == "spin up":
        tags[idx] = 1
        atoms.set_tags(tags)
    elif tool.value == "spin down":
        tags[idx] = 2
        atoms.set_tags(tags)
    elif tool.value == "remove atom":
        del atoms[idx]

    update_viewer()

In [ ]:
viewer = nglview.NGLWidget()
viewer.observe(on_picked, names='picked')
tool = ipw.Dropdown(description="On click...", options=["spin 0", "spin up", "spin down", "remove atom"])

display(viewer, tool)

## Step 3: Store structure in the AiiDA database

In [ ]:
def on_click_store(b):
    global atoms
    
    StructureData = DataFactory('structure')
    s = StructureData(ase=atoms)
    s.description = inp_descr.value
    s.store()
    print("Stored in AiiDA: "+repr(s))

    # TODO: create link to old structure

inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]))